# Get Swaps

In this notebook, we compile the data for borrow transactions from the API.

In [1]:
import requests
import json
import pandas as pd
import seaborn as sns
import cryptocompare
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from datetime import datetime
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import missingno as msno
from IPython.display import HTML
import matplotlib.dates as md
import math
import time

### Fetch Swap Transaction Data

Here, we write a query to fetch transaction information from the API.

In [2]:
lastId='""'
swap_data=[]
while(1):
    try:
        #set query
        query="""
        {
          swaps(first: 1000 orderBy: id where:{id_gt:"""+lastId+"""}) {
            pool{
                id
            }
            user{
                id
            }
            reserve{
                symbol
            }
            borrowRateModeFrom
            borrowRateModeTo
            stableBorrowRate
            variableBorrowRate
            timestamp
          }
        }
        """
        #make request
        url = 'https://api.thegraph.com/subgraphs/name/aave/protocol-v2'
        request = requests.post(url,json={'query':query})
        #store data
        swap_data.extend(request.json()['data']['swaps'])
        lastId = "\""+request.json()['data']['swaps'][-1]['id']+"\""
    except:
        #exit when no more data left to get
        break

#create borrows data frame
df_swaps = pd.DataFrame(swap_data)
df_swaps['type']='swap'
df_swaps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   borrowRateModeFrom  1000 non-null   object
 1   borrowRateModeTo    1000 non-null   object
 2   pool                1000 non-null   object
 3   reserve             1000 non-null   object
 4   stableBorrowRate    1000 non-null   object
 5   timestamp           1000 non-null   int64 
 6   user                1000 non-null   object
 7   variableBorrowRate  1000 non-null   object
 8   type                1000 non-null   object
dtypes: int64(1), object(8)
memory usage: 70.4+ KB


### Re-Format Swap Transaction Data

Some of the values in the dataset are dictionaries, so we must re-format the data to get the wanted field from those dictionaries.

In [3]:
#get id's
def getUser(row):
    return row['user']['id']
df_swaps['user']=df_swaps.apply(lambda x: getUser(x), axis=1)

def getPool(row):
    return row['pool']['id']
df_swaps['pool']=df_swaps.apply(lambda x: getPool(x), axis=1)

#get symbols
def getReserve(row):
    if not isinstance(row['reserve'],float):
        return row['reserve']['symbol']
    else:
        return np.nan
df_swaps['reserve']=df_swaps.apply(lambda x: getReserve(x), axis=1)

### Format Borrow Rates

Next, we format the borrow rates so that they are represented as percentages.

In [4]:
df_swaps['stableBorrowRate'] = df_swaps['stableBorrowRate'].astype(float)/1e27*100
df_swaps['variableBorrowRate'] = df_swaps['variableBorrowRate'].astype(float)/1e27*100

### Save Final Data Frame

Finally, we save the dataframe to a csv file.

In [5]:
df_swaps.to_csv("swaps.csv",index=False)